<a href="https://colab.research.google.com/github/nklsrmy/CCDEPLRL_EXERCISES_COM222ML/blob/main/Exercise8_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 8

Instructions: Finetune a model to answer the questions below.

In [1]:
!pip install --upgrade transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcs

In [2]:
!git config --global user.email "nics62796jauregui@gmail.com"
!git config --global user.name "nklsrmy"

In [3]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

## A. Model finetuning

In [4]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./distilbert-squad-checkpoint",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    save_steps=500,
    save_total_limit=2,
    fp16=True  # T4 supports this
)

In [6]:
from transformers import default_data_collator

data_collator = default_data_collator

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# put your answers here
def add_start_end_positions(example):
    inputs = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = example["answers"]
    start_char = answers["answer_start"][0]
    end_char = start_char + len(answers["text"][0])
    sequence_ids = inputs.sequence_ids()

    context_start = sequence_ids.index(1)
    context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

    token_start_index = context_start
    token_end_index = context_end

    while token_start_index < len(offset_mapping):
        if offset_mapping[token_start_index][0] <= start_char and offset_mapping[token_start_index][1] > start_char:
            break
        token_start_index += 1

    while token_end_index >= 0:
        if offset_mapping[token_end_index][0] < end_char and offset_mapping[token_end_index][1] >= end_char:
            break
        token_end_index -= 1

    inputs["start_positions"] = token_start_index
    inputs["end_positions"] = token_end_index
    return inputs

tokenized_squad = squad.map(add_start_end_positions, remove_columns=squad.column_names)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad,
    data_collator=data_collator,
    tokenizer=tokenizer
)

<ipython-input-10-b290c0d339d1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nics62796jauregui (nics62796jauregui-national-university-manila) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,2.007600


TrainOutput(global_step=626, training_loss=1.8260048814474965, metrics={'train_runtime': 131.6882, 'train_samples_per_second': 75.937, 'train_steps_per_second': 4.754, 'total_flos': 979898250240000.0, 'train_loss': 1.8260048814474965, 'epoch': 2.0})

In [12]:
trainer.save_model("distilbert-squad-model")
tokenizer.save_pretrained("distilbert-squad-model")

('distilbert-squad-model/tokenizer_config.json',
 'distilbert-squad-model/special_tokens_map.json',
 'distilbert-squad-model/vocab.txt',
 'distilbert-squad-model/added_tokens.json',
 'distilbert-squad-model/tokenizer.json')

#TEST

In [15]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-squad-model", tokenizer="distilbert-squad-model")

qa_data = [
    {
        "context": "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south",
        "question": "What is the largest ocean on Earth?",
        "expected": "Pacific Ocean"
    },
    {
        "context": "Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time",
        "question": "What was Isaac Newton's nationality?",
        "expected": "English"
    },
    {
        "context": "Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China.",
        "question": "where is Mount Everest located?",
        "expected": "in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China"
    },
    {
        "context": "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water",
        "question": "What do plants use to perform photosynthesis?",
        "expected": "Sunlight"
    },
    {
        "context": "Barack Obama served as the 44th president of the United States from 2009 to 2017.",
        "question": "When did Barack Obama serve as president?",
        "expected": "from 2009 to 2017"
    },
]

for i, example in enumerate(qa_data, 1):
    result = qa_pipeline(question=example["question"], context=example["context"])
    print(f"Q{i}: {example['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Expected Answer : {example['expected']}")
    print(f"Match           : {result['answer'].lower().strip() == example['expected'].lower().strip()}")
    print("-" * 50)


Device set to use cuda:0


Q1: What is the largest ocean on Earth?
Predicted Answer: Pacific Ocean
Expected Answer : Pacific Ocean
Match           : True
--------------------------------------------------
Q2: What was Isaac Newton's nationality?
Predicted Answer: physicist, astronomer
Expected Answer : English
Match           : False
--------------------------------------------------
Q3: where is Mount Everest located?
Predicted Answer: Himalayas
Expected Answer : in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China
Match           : False
--------------------------------------------------
Q4: What do plants use to perform photosynthesis?
Predicted Answer: sunlight
Expected Answer : Sunlight
Match           : True
--------------------------------------------------
Q5: When did Barack Obama serve as president?
Predicted Answer: 44th president of the United States from 2009 to 2017
Expected Answer : from 2009 to 2017
Match           : False
-----------------------------------------

In [16]:
!pip install evaluate --quiet

In [19]:
import evaluate
from transformers import pipeline

# Load the metric and pipeline
squad_metric = evaluate.load("squad")
qa_pipeline = pipeline("question-answering", model="distilbert-squad-model", tokenizer="distilbert-squad-model")

# Question set
qa_data = [
    {
        "id": "1",
        "context": "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south",
        "question": "What is the largest ocean on Earth?",
        "expected": "Pacific Ocean"
    },
    {
        "id": "2",
        "context": "Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time",
        "question": "What was Isaac Newton's nationality?",
        "expected": "English"
    },
    {
        "id": "3",
        "context": "Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China.",
        "question": "where is Mount Everest located?",
        "expected": "in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China"
    },
    {
        "id": "4",
        "context": "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water",
        "question": "What do plants use to perform photosynthesis?",
        "expected": "Sunlight"
    },
    {
        "id": "5",
        "context": "Barack Obama served as the 44th president of the United States from 2009 to 2017.",
        "question": "When did Barack Obama serve as president?",
        "expected": "from 2009 to 2017"
    }
]

predictions = []
references = []

for example in qa_data:
    result = qa_pipeline(question=example["question"], context=example["context"])
    predicted = result["answer"]
    expected = example["expected"]

    print(f"Q{i}: {example['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Expected Answer : {example['expected']}")

    predictions.append({
        "id": example["id"],
        "prediction_text": predicted
    })
    references.append({
        "id": example["id"],
        "answers": {"answer_start": [example["context"].find(expected)],
                    "text": [expected]}
    })

# Compute final score
results = squad_metric.compute(predictions=predictions, references=references)


Device set to use cuda:0


Q5: What is the largest ocean on Earth?
Predicted Answer: Pacific Ocean
Expected Answer : Pacific Ocean
Q5: What was Isaac Newton's nationality?
Predicted Answer: physicist, astronomer
Expected Answer : English
Q5: where is Mount Everest located?
Predicted Answer: Himalayas
Expected Answer : in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China
Q5: What do plants use to perform photosynthesis?
Predicted Answer: sunlight
Expected Answer : Sunlight
Q5: When did Barack Obama serve as president?
Predicted Answer: 44th president of the United States from 2009 to 2017
Expected Answer : from 2009 to 2017


## B. Question Set

1. Question 1

In [20]:
context1 = "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south"
question1 = "What is the largest ocean on Earth?"
# Expected Answer: Pacific Ocean

2. Question 2

In [21]:
context2 = "Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time"
question2 = "What was Isaac Newton's nationality?"
# Expected Answer: English

3. Question 3

In [22]:
context3 ="Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China."
question3 = "where is Mount Everest located?"
# Expected Answer: in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China

4. Question 4

In [23]:
context4 = "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water"
Question4 = "What do plants use to perform photosynthesis?"
# Expected Answer: Sunlight

5. Question 5

In [24]:
context5 = "Barack Obama served as the 44th president of the United States from 2009 to 2017."
question5 = "When did Barack Obama serve as president?"
# Expected Answer: from 2009 to 2017